In [ ]:
%pip install -Uqqq arize-phoenix-client arize-phoenix-otel openai requests openinference-instrumentation-openai

In [ ]:
import json
import os
from enum import Enum
from getpass import getpass
from itertools import chain
from textwrap import dedent

import openai
import pandas as pd
import requests
from IPython.display import HTML, display
from openai import OpenAI
from openai.lib._parsing import type_to_response_format_param
from openai.types.chat.completion_create_params import CompletionCreateParamsBase
from openinference.instrumentation.openai import OpenAIInstrumentor
from pydantic import BaseModel, create_model

from phoenix.client import Client
from phoenix.client.types import PromptVersion
from phoenix.otel import register

In [ ]:
tracer_provider = register()
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

In [ ]:
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key: ")

Install and start a local instance of Phoenix if needed.

In [ ]:
%pip install -Uqqq arize-phoenix
import phoenix as px

px.launch_app()

# Text Generation

## Quick Start

Here's a simple LLM invocation.

In [ ]:
params = CompletionCreateParamsBase(
    model="gpt-4o-mini",
    temperature=0,
    messages=[
        {"role": "system", "content": "You are coding poet."},
        {"role": "user", "content": "Write a haiku about recursion in programming."},
    ],
)
resp = OpenAI().chat.completions.create(**params)
print(resp.choices[0].message.content)

We can save the prompt in Phoenix.

In [ ]:
# prompt identifier should contain only alphanumeric characters, hyphens or underscores
prompt_identifier = "haiku-recursion"

prompt = Client().prompts.create(
    name=prompt_identifier,
    prompt_description="Haiku about recursion in programming",
    version=PromptVersion.from_openai(params),
)

We can fetch prompt from Phoenix.

In [ ]:
prompt = Client().prompts.get(prompt_identifier=prompt_identifier)
resp = OpenAI().chat.completions.create(**prompt.format())
print(resp.choices[0].message.content)

# Response Format

## Text summarization

Based on [this example](https://colab.research.google.com/github/openai/openai-cookbook/blob/a3e98ea4dcf866b5e7a3cb7d63dccaa68c7d63aa/examples/Structured_Outputs_Intro.ipynb#scrollTo=5eae3aea) from OpenAI cookbook.

In [ ]:
summarization_prompt = """\
    You will be provided with content from an article about an invention.
    Your goal will be to summarize the article following the schema provided.
    Here is a description of the parameters:
    - invented_year: year in which the invention discussed in the article was invented
    - summary: one sentence summary of what the invention is
    - inventors: array of strings listing the inventor full names if present, otherwise just surname
    - concepts: array of key concepts related to the invention, each concept containing a title and a description
    - description: short description of the invention
"""


class ArticleSummary(BaseModel):
    invented_year: int
    summary: str
    inventors: list[str]
    description: str

    class Concept(BaseModel):
        title: str
        description: str

    concepts: list[Concept]


response_format = type_to_response_format_param(ArticleSummary)

params = CompletionCreateParamsBase(
    model="gpt-4o-mini",
    temperature=0.2,
    messages=[
        {"role": "system", "content": dedent(summarization_prompt)},
        {"role": "user", "content": "{{text}}"},
    ],
    response_format=response_format,
)

Save prompt in Phoenix

In [ ]:
# prompt identifier should contain only alphanumeric characters, hyphens or underscores
prompt_identifier = "summarize-invention-article"

prompt = Client().prompts.create(
    name=prompt_identifier,
    prompt_description="Summarize an article about an invention",
    version=PromptVersion.from_openai(params),
)

In [ ]:
src = "https://raw.githubusercontent.com/openai/openai-cookbook/refs/heads/main/examples/data/structured_outputs_articles"
articles = [{"text": requests.get(f"{src}/{f}").text} for f in ["cnns.md", "llms.md", "moe.md"]]

Fetch prompt from Phoenix and apply to articles

In [ ]:
prompt = Client().prompts.get(prompt_identifier=prompt_identifier)


def get_response(input: dict[str, str]):
    response = OpenAI().chat.completions.create(**prompt.format(variables=input))
    return json.loads(response.choices[0].message.content)


# Collect results into a DataFrame.
res = pd.json_normalize(map(get_response, articles))
display(HTML(res.to_html()))

## UI generation

Based on [this example](https://platform.openai.com/docs/guides/structured-outputs?example=ui-generation).

In [ ]:
class _UIType(str, Enum):
    div = "div"
    button = "button"
    header = "header"
    section = "section"
    field = "field"
    form = "form"


class _Attribute(BaseModel):
    name: str
    value: str


class _UI(BaseModel):
    type: _UIType
    label: str
    children: list["_UI"]
    attributes: list["_Attribute"]


_UI.model_rebuild()

response_format = type_to_response_format_param(create_model("Response", ui=(_UI, ...)))

params = CompletionCreateParamsBase(
    messages=[{"role": "user", "content": "Generate form for {{feature}}"}],
    model="gpt-4o-mini",
    response_format=response_format,
)

Save prompt in Phoenix.

In [ ]:
# prompt identifier should contain only alphanumeric characters, hyphens or underscores
prompt_identifier = "ui-generator"

prompt = Client().prompts.create(
    name=prompt_identifier,
    version=PromptVersion.from_openai(params),
)

Fetch prompt from Phoenix.

In [ ]:
prompt = Client().prompts.get(prompt_identifier=prompt_identifier)

variables = {"feature": "user login"}
resp = OpenAI().chat.completions.create(**prompt.format(variables=variables))
print(resp.choices[0].message.content)

# Function Calling

## Entity extraction from user input

Based on [this example](https://colab.research.google.com/github/openai/openai-cookbook/blob/a3e98ea4dcf866b5e7a3cb7d63dccaa68c7d63aa/examples/Structured_Outputs_Intro.ipynb#scrollTo=ee802699) from OpenAI cookbook.

In [ ]:
product_search_prompt = """\
    You are a clothes recommendation agent, specialized in finding the perfect match for a user.
    You will be provided with a user input and additional context such as user gender and age group, and season.
    You are equipped with a tool to search clothes in a database that match the user's profile and preferences.
    Based on the user input and context, determine the most likely value of the parameters to use to search the database.

    Here are the different categories that are available on the website:
    - shoes: boots, sneakers, sandals
    - jackets: winter coats, cardigans, parkas, rain jackets
    - tops: shirts, blouses, t-shirts, crop tops, sweaters
    - bottoms: jeans, skirts, trousers, joggers

    There are a wide range of colors available, but try to stick to regular color names.
"""


class ProductSearchParameters(BaseModel):
    class Category(str, Enum):
        shoes = "shoes"
        jackets = "jackets"
        tops = "tops"
        bottoms = "bottoms"

    category: Category
    subcategory: str
    color: str


params = CompletionCreateParamsBase(
    model="gpt-4o-mini",
    temperature=0,
    messages=[
        {"role": "system", "content": dedent(product_search_prompt)},
        {
            "role": "user",
            "content": "CONTEXT: {{context}}\n\nUSER INPUT: {{user_input}}",
        },
    ],
    tools=[
        openai.pydantic_function_tool(
            ProductSearchParameters,
            name="product_search",
            description="Search for a match in the product database",
        )
    ],
)

Save prompt in Phoenix

In [ ]:
# prompt identifier should contain only alphanumeric characters, hyphens or underscores
prompt_identifier = "extract-email-addresses-into-json-data"

prompt = Client().prompts.create(
    name=prompt_identifier,
    prompt_description="Extract email addresses into JSON data",
    version=PromptVersion.from_openai(params),
)

Define example inputs

In [ ]:
example_inputs = [
    {
        "user_input": "I'm looking for a new coat. I'm always cold so please something warm! Ideally something that matches my eyes.",
        "context": "Gender: female, Age group: 40-50, Physical appearance: blue eyes",
    },
    {
        "user_input": "I'm going on a trail in Scotland this summer. It's goind to be rainy. Help me find something.",
        "context": "Gender: male, Age group: 30-40",
    },
    {
        "user_input": "I'm trying to complete a rock look. I'm missing shoes. Any suggestions?",
        "context": "Gender: female, Age group: 20-30",
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: summer",
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: winter",
    },
    {
        "user_input": "Can you help me find a dress for a Barbie-themed party in July?",
        "context": "Gender: female, Age group: 20-30",
    },
]

Fetch prompt from Phoenix and apply to example inputs

In [ ]:
prompt = Client().prompts.get(prompt_identifier=prompt_identifier)


def get_response(input: dict[str, str]):
    response = OpenAI().chat.completions.create(**prompt.format(variables=input))
    tool_calls = response.choices[0].message.tool_calls
    return ({**json.loads(tc.function.arguments), **input} for tc in tool_calls)


# Collect results into a DataFrame.
res = pd.json_normalize(chain.from_iterable(map(get_response, example_inputs)))
res = res.set_index(["user_input", "context"])
display(HTML(res.to_html()))

# Reasoning

## Using Reasoning for Routine Genaration

Based on [this example](https://colab.research.google.com/github/openai/openai-cookbook/blob/a3e98ea4dcf866b5e7a3cb7d63dccaa68c7d63aa/examples/o1/Using_reasoning_for_routine_generation.ipynb#scrollTo=ReYoD4FaaPJg&line=8&uniqifier=1) from OpenAI cookbook

In [ ]:
CONVERSION_PROMPT = """\
You are a helpful assistant tasked with taking an external facing help center article and converting it into a internal-facing programmatically executable routine optimized for an LLM.
The LLM using this routine will be tasked with reading the policy, answering incoming questions from customers, and helping drive the case toward resolution.

Please follow these instructions:
1. **Review the customer service policy carefully** to ensure every step is accounted for. It is crucial not to skip any steps or policies.
2. **Organize the instructions into a logical, step-by-step order**, using the specified format.
3. **Use the following format**:
   - **Main actions are numbered** (e.g., 1, 2, 3).
   - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b).
      **Sub-actions should start on new lines**
   - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the product was purchased within 30 days, then...').
   - **For instructions that require more information from the customer**, provide polite and professional prompts to ask for additional information.
   - **For actions that require data from external systems**, write a step to call a function using backticks for the function name (e.g., `call the check_delivery_date function`).
      - **If a step requires the customer service agent to take an action** (e.g., process a refund), generate a function call for this action (e.g., `call the process_refund function`).
      - **Define any new functions** by providing a brief description of their purpose and required parameters.
   - **If there is an action an assistant can performon behalf of the user**, include a function call for this action (e.g., `call the change_email_address function`), and ensure the function is defined with its purpose and required parameters.
      - This action may not be explicitly defined in the help center article, but can be done to help the user resolve their inquiry faster
   - **The step prior to case resolution should always be to ask if there is anything more you can assist with**.
   - **End with a final action for case resolution**: calling the `case_resolution` function should always be the final step.
4. **Ensure compliance** by making sure all steps adhere to company policies, privacy regulations, and legal requirements.
5. **Handle exceptions or escalations** by specifying steps for scenarios that fall outside the standard policy.

**Important**: If at any point you are uncertain, respond with "I don't know."

Please convert the customer service policy into the formatted routine, ensuring it is easy to follow and execute programmatically.\
"""

params = CompletionCreateParamsBase(
    model="o3-mini",
    messages=[
        {"role": "user", "content": dedent(CONVERSION_PROMPT)},
        {"role": "user", "content": "POLICY:\n\n{{content}}"},
    ],
)

Save prompt in Phoenix.

In [ ]:
# prompt identifier should contain only alphanumeric characters, hyphens or underscores
prompt_identifier = "convert-customer-service-policy"

prompt = Client().prompts.create(
    name=prompt_identifier,
    prompt_description="Convert customer service policy into a routine",
    version=PromptVersion.from_openai(params),
)

Download articles.

In [ ]:
url = "https://raw.githubusercontent.com/openai/openai-cookbook/a3e98ea4dcf866b5e7a3cb7d63dccaa68c7d63aa/examples/data/helpcenter_articles.csv"
articles = pd.read_csv(url).sample(1)
display(HTML(articles.to_html()))

Fetch prompt from Phoenix and apply to data.

In [ ]:
prompt = Client().prompts.get(prompt_identifier=prompt_identifier)


def process_article(input: dict[str, str]):
    resp = OpenAI().chat.completions.create(**prompt.format(variables=input))
    routine = resp.choices[0].message.content
    return {"policy": input["policy"], "content": input["content"], "routine": routine}


# Collect results into a DataFrame.
res = pd.json_normalize(map(process_article, articles.to_dict(orient="records")))
display(HTML(res.to_html()))